# This is the code I will be building for my thesis project that I am writing at the chair of Public Economics of Prof. Daniel Schunk
# with Dr. Katharina Hartinger as my adviser.

I will be documenting everything using these markdown cells to describe ideas, processes and problems that arise while writing the code. 

# This is the code I will be building for my thesis project that I am writing at the chair of Public Economics of Prof. Daniel Schunk
# with Dr. Katharina Hartinger as my adviser.

I will be documenting everything using these markdown cells to describe ideas, processes and problems that arise while writing the code.

All the packages that will be imported to run the code for my thesis will be readable in the cell below. I have declared a variety of dataframes and csv to keep track of every change I have made to the original dataset, allowing me to closely check whether the algorithm worked as intended. Users who are experienced with the dataset can of course drop/ignore these dataframes.

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
import enchant
import re


In [2]:
df = pd.read_csv("Game_data_05_08_24.csv")  

# Create the 'Success' column
df['Success'] = df.apply(lambda row: 1 if row['Guess'] == row['Mystery Word'] else 0, axis=1)

df.head()

,Table Number,Move,Clues,Mystery Word,Guess,Mode,Speed,Language,End,Success
0,312175859,1609,"Koperta, Naklejony, Pocztowy, Pocztówka",Stamp,Znaczek,NaN,NaN,NaN,NaN,0
1,312175859,1615,"W, Rimmikub, Glazura, Kwadracik",Tile,Płytka,NaN,NaN,NaN,NaN,0
2,312175859,1626,"Grajek, Baśń, Ta, Tuj",Rat,Bard,NaN,NaN,NaN,NaN,0
3,312175859,1632,"Płacony, Restauracja, Kid, Western",Bill,Tip,NaN,NaN,NaN,NaN,0
4,312175859,1638,"Władzy, Złoty, Tytanowy, Obrączka",Ring,Ring,NaN,NaN,NaN,NaN,1


Hier schreibe ich alles auf, was ich an dem Datensatz irgendwie verändere bzw. welche Probleme ich versuche zu lösen:

1. Das Sprachenproblem: ganz viele Runden haben leider keine Klassifizierung, in welcher Sprache sie gespielt wurden. Das ist insofern problematisch, da ich ja auch die Sprache als potenziellen Prädiktor nutzen möchte. Daher möchte ich für alle Runden die Sprache kennen. Wie finde ich also heraus, in welcher Sprache gespielt wurde?

**Lösungsansatz:**
Ich lasse Wörterbücher über Mystery Word, Guess und Clues laufen, um zunächst alle Sprachen zu erkennen. Dann vergleiche ich die erkannten Sprachen- wenn diese übereinstimmen, wird die Rundensprache auf das entsprechende Match festgelegt. Ein Beispiel: das Mystery Word ist 'Bathroom', der Guess 'Toilet' und die Clues sind 'Morning', 'Shower', 'Sink' und 'Towel'. Hierbei sollte der Algorithmus jetzt feststellen, dass alle Wörter in der englischen Sprache existieren und somit die Rundensprache auf Englisch setzen. 

**Probleme:** 
1. Die Wörterbücher erkennen viele Dinge nicht bzw. falsch. Gerade polnische Wörter sind für den Algorithmus schwierig. 
2. Die Wörter existieren in mehreren Sprachen. Z.B. das Wort 'Bad' existiert sowohl im Deutschen als auch im Englischen. Der Algorithmus erkennt hier insofern ein Match sowohl im deutschen als auch im englichen Wörterbuch. Das kann natürlich zu Problemen führen. Daher vergleicht der Algorithmus jeweils immer unabhängig- es ist nicht schlimm, wenn ein Wort in mehreren Sprachen existiert, solang eine der Sprachen über alle Kategorien matcht. 
3. 

2. Das Sprachenproblem, fortgesetzt: für spätere Analysen, besonders im Rahmen des Originalitätsmaßes, ist es wichtig, dass alle Wörter in der gleichen Sprache, also der Rundensprache sind. Denn logischerweise wäre ein Wort in französischer Sprache sehr weit entfernt von einem in der deutschen Sprache- selbst wenn die Bedeutung gar nicht so weit voneinander entfernt ist. Daher müssen tatsächlich alle Wörter, die nicht in der Rundensprache existieren, aussortiert werden- und ihre Runden der Konsistenz halber gleich mit. Um das zu erreichen, werde ich eine zweite Runde an Checks einbauen, die, sobald eine Sprache festgelegt wurde, die Clues erneut überprüft, und im Fall, dass ein Wort nicht im Wörterbuch dieser Sprache existiert, die Runde entfernt. 

Hierbei muss ich aber noch schauen, wieviele Datenpunkte mir hinterher übrig bleiben. 

## Documentation for Removing Rounds with NaN Values and Storing Them Separately

### Overview
This script is designed to clean a dataset by identifying and removing rounds (rows) that contain missing values (`NaN`) in key columns. The removed rounds are stored in a separate dataset for potential future analysis or review. The script also provides a summary of how many rounds were retained and how many were removed.

### Purpose
The cleaning process focuses on four key columns:
1. `Mode`
2. `Speed`
3. `Language`
4. `End`

Rounds with missing values in any of these columns are considered incomplete and are removed from the main dataset. The removed rounds are stored in a separate dataset, allowing for a clear distinction between complete and incomplete data.

### Code Breakdown

1. **Loading the Dataset**:
   - The dataset is loaded into a pandas DataFrame for processing.

2. **Identifying and Separating Incomplete Rounds**:
   - The script checks for missing (`NaN`) values in the specified columns (`Mode`, `Speed`, `Language`, `End`).
   - Rounds with any `NaN` values in these columns are separated into a new DataFrame (`removed_rounds`).
   - The remaining rounds, which are complete, are stored in another DataFrame (`cleaned_data`).

3. **Counting the Rounds**:
   - The script calculates and prints the number of rounds that were retained (i.e., those without `NaN` values).
   - It also calculates and prints the number of rounds that were removed due to missing values.

4. **Output**:
   - The cleaned dataset is ready for further analysis, while the removed rounds are preserved for any necessary follow-up.


In [4]:
# Separate the rounds with NaN values in 'Mode', 'Speed', 'Language', or 'End' columns
removed_rounds = df[df[['Mode', 'Speed', 'Language', 'End']].isna().any(axis=1)]
cleaned_data = df.dropna(subset=['Mode', 'Speed', 'Language', 'End'])

# Count the number of rounds
total_rounds = df.shape[0]
cleaned_rounds_count = cleaned_data.shape[0]
removed_rounds_count = removed_rounds.shape[0]

# Print the counts
cleaned_rounds_count, removed_rounds_count

(14035, 8373)

# Documentation: Removing Rounds Based on Clue Content

## Overview
This script is designed to clean a dataset by identifying and removing entire rounds (rows) of data where any clue contains special characters, numbers, or multiple words (indicated by spaces). The script also saves the removed rounds to a separate CSV file for further analysis or review.

## Purpose
The goal of this script is to ensure the integrity of your dataset by removing any rounds that could skew the analysis. Specifically, it addresses the following issues:
1. **Special Characters**: Any clue containing characters other than letters, numbers, underscores, or spaces is flagged.
2. **Numbers**: Clues containing numeric digits are flagged.
3. **Multiple Words**: Clues containing more than one word (indicated by spaces) are flagged.

If any clue in a round meets any of these criteria, the entire round is removed from the dataset.

## Code Breakdown

### 1. **Copying the Dataset**
   - `df_cleaned = cleaned_data.copy()`
   - This creates a copy of the original dataset to avoid modifying it directly. All cleaning operations are performed on this copy.

### 2. **Initialization**
   - The script initializes dictionaries to store the counts and lists of removed words based on the specific reason for their removal:
     - `special_characters`
     - `numbers`
     - `spaces`

### 3. **Defining the `clean_rounds` Function**
   - This function iterates over each row in the dataset and checks each clue in the 'Clues' column:
     - **Special Characters**: Uses the regular expression `r'[^\w\s]'` to identify and flag any special characters.
     - **Numbers**: Uses `r'\d'` to detect any numeric digits in the clues.
     - **Spaces**: Checks for spaces to determine if the clue consists of multiple words.
   - If any clue in a round meets the criteria, the round is marked for deletion.
   - The removed rounds are saved to a separate DataFrame.

### 4. **Applying the `clean_rounds` Function**
   - `df_cleaned, removed_rounds = clean_rounds(df_cleaned, 'Clues')`
   - The function is applied to the dataset, cleaning it by removing the identified rounds and storing those rounds separately.

### 5. **Saving the Results**
   - The cleaned dataset and the removed rounds are saved to separate CSV files:
     - `'cleaned_data_with_cleaned_clues.csv'` for the cleaned data.
     - `'removed_rounds.csv'` for the rounds that were removed.

### 6. **Summary Output**
   - The script prints a summary of the number of words (or clues) that were flagged and removed for each category (`special_characters`, `numbers`, `spaces`).

In [25]:
# Create a copy of the dataset to avoid modifying the original data
df_cleaned = cleaned_data.copy()

# Initialize dictionaries to store removed words counts and lists
removed_words_counts = {
    'special_characters': 0,
    'numbers': 0,
    'spaces': 0
}

removed_words_lists = {
    'special_characters': [],
    'numbers': [],
    'spaces': []
}

# Function to clean entire rounds if any clue contains undesired characters
def clean_rounds(dataframe, column_name):
    rows_to_drop = []
    for index, clues_string in dataframe[column_name].items():
        clues_list = str(clues_string).split(',')
        for clue in clues_list:
            original_clue = clue.strip()
            
            # Check for special characters
            if re.search(r'[^\w\s]', original_clue):
                removed_words_lists['special_characters'].append(original_clue)
                removed_words_counts['special_characters'] += 1
                rows_to_drop.append(index)
                break  # Skip further checks and mark the round for deletion
            
            # Check for numbers
            if re.search(r'\d', original_clue):
                removed_words_lists['numbers'].append(original_clue)
                removed_words_counts['numbers'] += 1
                rows_to_drop.append(index)
                break  # Skip further checks and mark the round for deletion
            
            # Check for spaces (multiple words)
            if ' ' in original_clue:
                removed_words_lists['spaces'].append(original_clue)
                removed_words_counts['spaces'] += 1
                rows_to_drop.append(index)
                break  # Skip further checks and mark the round for deletion

    # Save the removed rounds to a separate DataFrame
    removed_rounds = dataframe.loc[rows_to_drop]
    
    # Drop the identified rows from the original DataFrame
    dataframe = dataframe.drop(rows_to_drop)

    return dataframe, removed_rounds

# Apply the function to delete rounds instead of just words
df_cleaned, removed_rounds = clean_rounds(df_cleaned, 'Clues')

# Save the cleaned DataFrame and removed rounds DataFrame to CSV files
df_cleaned.to_csv('cleaned_data_with_cleaned_clues.csv', index=False)
removed_rounds.to_csv('removed_rounds.csv', index=False)

# Print out the counts of removed words for each category
print("Summary of Removed Words:")
print(f" - Special Characters: {removed_words_counts['special_characters']}")
print(f" - Numbers: {removed_words_counts['numbers']}")
print(f" - Spaces (multiple words): {removed_words_counts['spaces']}")
print(f" - Total Removed Words: {sum(removed_words_counts.values())}")

print("\nFiles have been saved:")
print(" - Cleaned data with cleaned clues: 'cleaned_data_with_cleaned_clues.csv'")
print(" - Removed rounds: 'removed_rounds.csv'")



Summary of Removed Words:
 - Special Characters: 1100
 - Numbers: 319
 - Spaces (multiple words): 0
 - Total Removed Words: 1419

Files have been saved:
 - Cleaned data with cleaned clues: 'cleaned_data_with_cleaned_clues.csv'
 - Removed rounds: 'removed_rounds.csv'


Number of individual clues with quotation marks in the cleaned dataset: 0
Number of individual clues with quotation marks in the cleaned dataset: 0


# Documentation: Dictionary Validation and Clue Processing

## Overview
This script processes a dataset containing clues in multiple languages, validating each clue against language-specific dictionaries. It categorizes rounds based on the percentage of clues that are valid according to the dictionary and saves the results in different CSV files. The script also counts and prints the number of rows for each language, specifically focusing on French in this case.

## Purpose
The goal of this script is to clean and validate a dataset of clues by ensuring that they exist in the appropriate language dictionary. The script categorizes the rounds based on the percentage of valid clues and generates corresponding CSV files for further analysis.

## Code Breakdown

### 1. **Loading Dictionaries**
   - The function `load_dictionary(file_name)` loads a dictionary file and converts the list of words into a lowercase set for efficient lookup.
   - Dictionaries for English (`words.txt`), German (`vocab_german.txt`), and French (`francais.txt`) are loaded. The French dictionary came from this Git: https://github.com/Taknok/French-Wordlist/blob/master/francais.txt

### 2. **Loading the Cleaned Dataset**
   - The dataset is loaded from a CSV file named `cleaned_data_with_cleaned_clues.csv` into the DataFrame `df_cleaned_with_cleaned_clues`.

### 3. **Validating Clues Against Dictionaries**
   - The function `check_clues_against_dictionary(dataframe, clues_column, language_column)` iterates through each round in the dataset and validates clues against the appropriate dictionary based on the language of the round.
   - **Categorization**:
     - **Fully Valid Rounds**: Rounds where all clues are valid.
     - **At Least 75% Valid Rounds**: Rounds where at least 75% of the clues are valid.
     - **At Least 50% Valid Rounds**: Rounds where at least 50% of the clues are valid.
     - **Invalid Rounds**: Rounds where fewer than 50% of the clues are valid.
   - The function returns four DataFrames corresponding to these categories.

### 5. **Saving the Results**
   - The categorized DataFrames are saved as separate CSV files:
     - `fully_valid_clues.csv`: Contains only rounds where every clue is valid.
     - `atleast_75_valid_clues.csv`: Contains rounds where at least 75% of clues are valid.
     - `atleast_50_valid_clues.csv`: Contains rounds where at least 50% of clues are valid.
     - `invalid_rounds_due_to_dictionary.csv`: Contains rounds where fewer than 50% of the clues are valid.

### 6. **Printing the Summary**
   - The script prints a summary of the number of rounds in each category:
     - **Rounds where every clue is valid**: Count of fully valid rounds.
     - **Rounds where at least 75% of clues are valid**: Count of rounds with at least 75% valid clues.
     - **Rounds where at least 50% of clues are valid**: Count of rounds with at least 50% valid clues.
     - **Rounds rejected due to invalid clues**: Count of rounds with fewer than 50% valid clues.
   - The total number of rows in the dataset is also printed.

### 7. **Output Files**
   - **`fully_valid_clues.csv`**: Rounds where all clues are valid.
   - **`atleast_75_valid_clues.csv`**: Rounds with at least 75% valid clues.
   - **`atleast_50_valid_clues.csv`**: Rounds with at least 50% valid clues.
   - **`invalid_rounds_due_to_dictionary.csv`**: Rounds with less than 50% valid clues.

In [44]:
# Load dictionaries for each language
def load_dictionary(file_name):
    with open(file_name, 'r') as file:
        return set(word.strip().lower() for word in file.readlines())

# Load the dictionaries (adjust the file paths as necessary)
english_words = load_dictionary('words.txt')
german_words = load_dictionary('vocab_german.txt')
french_words = load_dictionary('francais.txt')

# Function to check if clues exist in the respective language's dictionary
def check_clues_against_dictionary(dataframe, clues_column, language_column):
    all_valid_rows = []
    atleast_75_valid_rows = []
    atleast_50_valid_rows = []
    invalid_rows = []

    for index, row in dataframe.iterrows():
        clues_list = str(row[clues_column]).split(',')
        language = row[language_column].strip().lower()

        # Select the correct dictionary based on the language of the round
        if language == 'english':
            dictionary = english_words
        elif language == 'deutsch':
            dictionary = german_words
        elif language == 'français':
            dictionary = french_words 
        else:
            continue  # Skip if the language is not recognized
        
        valid_clues = [clue.strip().lower() for clue in clues_list if clue.strip().lower() in dictionary]
        total_clues = len(clues_list)
        valid_clues_count = len(valid_clues)

        # Categorize the row based on the percentage of valid clues
        if valid_clues_count == total_clues:
            all_valid_rows.append(index)
            atleast_75_valid_rows.append(index)
            atleast_50_valid_rows.append(index)
        elif valid_clues_count >= 0.75 * total_clues:
            atleast_75_valid_rows.append(index)
            atleast_50_valid_rows.append(index)
        elif valid_clues_count >= 0.50 * total_clues:
            atleast_50_valid_rows.append(index)
        else:
            invalid_rows.append(index)

    # Create DataFrames based on the indices
    df_all_valid = dataframe.loc[all_valid_rows]
    df_atleast_75_valid = dataframe.loc[atleast_75_valid_rows]
    df_atleast_50_valid = dataframe.loc[atleast_50_valid_rows]
    df_invalid = dataframe.loc[invalid_rows]

    return df_all_valid, df_atleast_75_valid, df_atleast_50_valid, df_invalid

# Assuming df_cleaned is your DataFrame after the previous cleaning steps
df_all_valid, df_atleast_75_valid, df_atleast_50_valid, df_invalid = check_clues_against_dictionary(df_cleaned_with_cleaned_clues, 'Clues', 'Language')

# Save the DataFrames to CSV files
df_all_valid.to_csv('fully_valid_clues.csv', index=False)
df_atleast_75_valid.to_csv('atleast_75_valid_clues.csv', index=False)
df_atleast_50_valid.to_csv('atleast_50_valid_clues.csv', index=False)
df_invalid.to_csv('invalid_rounds_due_to_dictionary.csv', index=False)

# Print out the counts of rounds in each category
print("Summary of Rounds Based on Dictionary Validation:")
print(f" - Rounds where every clue is valid: {len(df_all_valid)}")
print(f" - Rounds where at least 75% of clues are valid: {len(df_atleast_75_valid)}")
print(f" - Rounds where at least 50% of clues are valid: {len(df_atleast_50_valid)}")
print(f" - Rounds rejected due to invalid clues: {len(df_invalid)}")
print(f"Total number of rows in the dataset: {len(df_cleaned_with_cleaned_clues)}")

print("\nFiles have been saved:")
print(" - Fully valid clues: 'fully_valid_clues.csv'")
print(" - At least 75% valid clues: 'atleast_75_valid_clues.csv'")
print(" - At least 50% valid clues: 'atleast_50_valid_clues.csv'")
print(" - Invalid rounds: 'invalid_rounds_due_to_dictionary.csv'")

Number of rows with 'French' as the language: 2941
Summary of Rounds Based on Dictionary Validation:
 - Rounds where every clue is valid: 4965
 - Rounds where at least 75% of clues are valid: 8246
 - Rounds where at least 50% of clues are valid: 11050
 - Rounds rejected due to invalid clues: 1566
Total number of rows in the dataset: 12616

Files have been saved:
 - Fully valid clues: 'fully_valid_clues.csv'
 - At least 75% valid clues: 'atleast_75_valid_clues.csv'
 - At least 50% valid clues: 'atleast_50_valid_clues.csv'
 - Invalid rounds: 'invalid_rounds_due_to_dictionary.csv'
